In [21]:
%cd /N/dc2/projects/lifebid/Paolo/code/re_dwi/sandbox

/N/dc2/projects/lifebid/Paolo/code/re_dwi/sandbox


In [81]:
sub = '00'

#### Setting Filenames

In [83]:
dicom_dwi = 'input/sub-%s/dwi' % sub
dicom_t1w = 'input/sub-%s/anat' % sub

dwi_dir = 'sub-%s/dwi' % sub
dwi_src = 'sub-%s_dwi' % sub
!mkdir -p {dwi_dir}

dwi_pre = 'derivatives/preprocess/sub-%s' % sub
!mkdir -p {dwi_pre}

b0_src = 'sub-%s_var-b0src_dwi' % sub
b0_otsu = 'sub-%s_var-b0otsu_dwi' % sub
dwi_nodif = 'sub-%s_var-otsu_dwi_mask' % sub
dwi_ecc = 'sub-%s_var-ecc_dwi' % sub
b0_ecc = 'sub-%s_var-b0ecc_dwi' % sub

t1w_dir = 'sub-%s/anat' % sub
t1w_src = 'sub-%s_T1w.nii.gz' % sub
!mkdir -p {t1w_dir}

t1w_pre = 'derivatives/preprocess/sub-%s' % sub
!mkdir -p {t1w_pre}
t1w_bet = 'sub-%s_var-bet_T1w.nii.gz' % sub
t1w_mas = 'sub-%s_var-bet_T1w_mask.nii.gz' % sub
t1w_5tt = 'sub-%s_var-fsl_dtissue.nii.gz' % sub

fs_dir = 'derivatives/freesurfer/sub-%s' % sub
!mkdir -p {fs_dir}
fs_mas = 'sub-%s_var-fs_T1w_mask.nii.gz' % sub
fs_t1w = 'sub-%s_var-fs_T1w.nii.gz' % sub

t1w_fsl_wmm = 'sub-%s_var-fsl_space-T1w_class-wm_dtissue.nii.gz' % sub
b0_fsl_omat = 'sub-%s_var-fsl_affine.mat' % sub
dwi_fsl_wmm = 'sub-%s_var-fsl_space-dwi_class-wm_dtissue.nii.gz' % sub


#### Import and convert diffusion data

In [5]:
tmpdir = '_tmp_dcm2nii'
!mkdir -p {tmpdir}

!dcm2nii -f y -e n -p n -a n -d n -g n -i n -o {tmpdir} {dicom_dwi}

fname = !basename {tmpdir}/*.bval
fname = fname[0].strip('.bval')
!mv {tmpdir}/{fname}.nii {dwi_dir}/{dwi_src}.nii
!gzip -f {dwi_dir}/{dwi_src}.nii
!mv {tmpdir}/{fname}.bval {dwi_dir}/{dwi_src}.bvals
!mv {tmpdir}/{fname}.bvec {dwi_dir}/{dwi_src}.bvecs
!rm -r {tmpdir}

Chris Rorden's dcm2nii :: 1JUNE2015 64bit BSD License
reading preferences file /N/u/pavesani/Karst/.dcm2nii/dcm2nii.ini
Data will be exported to _tmp_dcm2nii/
Validating 1750 potential DICOM images.
Found 1750 DICOM images.
Converting 1750/1750  volumes: 35
1.3.12.2.1107.5.2.13.18805.30000017101606244498400005868.DCM->13122110752131880530000017101606244498400005868.nii
For slice timing correction: the slice order is Siemens-even interleaved ascending (2,4..,1,3...)
 brightest voxel was 4095: data will be saved as 16-bit signed integer.
Saving _tmp_dcm2nii/13122110752131880530000017101606244498400005868.nii
Number of diffusion directions = 35


In [6]:
!mrconvert {dwi_dir}/{dwi_src}.nii.gz {dwi_dir}/{dwi_src}.mif \
    -fslgrad {dwi_dir}/{dwi_src}.bvecs {dwi_dir}/{dwi_src}.bvals \
    -datatype float32 -stride 0,0,0,1 -nthreads 5 -force -quiet

Diffusion brain mask

In [65]:
!dwiextract  {dwi_dir}/{dwi_src}.mif - -shell 0 -quiet | mrmath - mean {dwi_pre}/{b0_src}.mif -axis 3 -force -quiet 
!mrconvert {dwi_pre}/{b0_src}.mif {dwi_pre}/{b0_src}.nii.gz -force -quiet

In [67]:
import cmd_median_otsu as cmo
mo_src = dwi_pre + '/' + b0_src + '.nii.gz'
mo_out = dwi_pre + '/' + b0_otsu + '.nii.gz'
mo_mask = dwi_pre + '/' + dwi_nodif + '.nii.gz'
cmo.cmd_median_otsu(mo_src, mo_out, mo_mask)
!mrconvert {dwi_pre}/{b0_otsu}.nii.gz {dwi_pre}/{b0_otsu}.mif -force -quiet
!mrconvert {dwi_pre}/{dwi_nodif}.nii.gz {dwi_pre}/{dwi_nodif}.mif -force -quiet

Eddy correction

In [ ]:
!dwipreproc AP {dwi_dir}/{dwi_src}.mif {dwi_pre}/{dwi_ecc}.mif -rpe_none -force -quiet

In [ ]:
!dwiextract  {dwi_dir}/{dwi_src}.mif - -shell 0 -quiet | mrmath - mean {dwi_pre}/{b0_ecc}.mif -axis 3 -force -quiet
!mrconvert {dwi_pre}/{b0_ecc}.mif {dwi_pre}/{b0_ecc}.nii.gz -force -quiet

#### Import structural data

In [36]:
!cp {dicom_t1w}/{t1w_src} {t1w_dir}/{t1w_src}

#### Brain extraction with 'bet'

In [42]:
!bet {t1w_dir}/{t1w_src} {t1w_pre}/{t1w_bet} -R -m

#### Brain structure segmentation with FSL

In [45]:
!5ttgen fsl {t1w_pre}/{t1w_bet} {t1w_pre}/{t1w_5tt} -premasked -nocrop -force -quiet

5ttgen: [WARNING] Output file sub-00_var-fsl_dtissue.nii.gz already exists; will be overwritten at script completion


#### Brain structure segmentation with FREESURFER

In [ ]:
!SUBJECTS_DIR={fs_dir}; recon-all -all -i {t1w_dir}/{t1w_src} -s freesurfer -parallel

#### White Matter mask segmentation with FSL

In [48]:
tmpdir = '_tmp_wm'
!mkdir -p {tmpdir}
!fslsplit {wdir}/{tt5} {tmpdir}/wm_ -t 
!mv {tmpdir}/wm_0002.nii.gz {t1w_pre}/{t1w_fsl_wmm}
!rm -r {tmpdir}

In [70]:
!flirt -in {t1w_pre}/{t1w_fsl_wmm} \
    -ref {dwi_pre}/{b0_ecc} \
    -omat {t1w_pre}/{b0_fsl_omat} 
!flirt -in {tiw_pre}/{t1w_fsl_wmm} \
    -ref {dwi_pre}/{b0_ecc} \
    -applyxfm -init {t1w_pre}/{b0_fsl_omat} \
    -out {t1w_pre}/{dwi_fsl_wmm} \
    -interp nearestneighbour

#### White Matter mask segmentation with FREESURFER

In [ ]:
from wm_from_aparc_aseg import wm_from_aparc_aseg
wdir = 'derivatives/preprocess/sub-%s/anat' % sub
t1w_wmm = t1w_pre + '/' + 'sub-%s_var-fs_space-T1w_class-wm_dtissue.nii.gz' % sub
aparc_aseg = fs_dir + '/' + 
wm_from_aparc_aseg(aparc_aseg, t1w_wmm)

In [ ]:
omat = 'sub-%s_var-fs_affine.mat' % sub
dwi_wmm = 'sub-%s_var-fs_space-dwi_class-wm_dtissue.nii.gz' % sub
b0 = 'sub-%s/dwi/sub-%s_var-b0_dwi.nii.gz' % (sub, sub)
!flirt -in {wdir}/{t1w_wmm} \
    -ref {b0} \
    -omat {wdir}/{omat} 
!flirt -in {wdir}/{t1w_wmm} \
    -ref {b0} \
    -applyxfm -init {wdir}/{omat} \
    -out {wdir}/{dwi_wmm} \
    -interp nearestneighbour

In [62]:
dname = 'derivatives/reconstruction/sub-%s' % sub
dwi_dir = 'sub-%s/dwi' % sub
dwi_src = 'sub-%s_dwi.mif' % sub
wmm_dir = 'derivatives/preprocess/sub-%s/anat' % sub
wmm_src = 'sub-%s_var-fsl_space-T1w_class-wm_dtissue.nii.gz' % sub
dti = 'sub-%s_var-mr3_DTI.mif' % sub
fa = 'sub-%s_var-mr3_FA.mif' % sub
sf = 'sub-%s_var-mr3_SF.mif' % sub
rf = 'sub-%s_var-mr3_response.txt' % sub
odf = 'sub-%s_var-mr3_ODF.mif' % sub
!mkdir -p {dname}

In [63]:
!dwi2tensor {dwi_dir}/{dwi_src} {dname}/{dti} 
!tensor2metric -fa {dname}/{fa} {dname}/{dti} -mask {wmm_dir}/{wmm_src}
!maskfilter {wmm_dir}/{wmm_src} erode -npass 3 - | \
    mrmath {wmm_dir}/{wmm_src} - product - | \
    mrthreshold - -abs 0.7 {dname}/{sf}

dwi2tensor: [100%] computing tensors
tensor2metric: [100%] uncompressing image "derivatives/preprocess/sub-00/anat/sub-00_var-fsl_space-T1w_class-wm_dtissue.nii.gz"
tensor2metric: [ERROR] dimension mismatch between "derivatives/reconstruction/sub-00/sub-00_var-mr3_DTI.mif" and "derivatives/preprocess/sub-00/anat/sub-00_var-fsl_space-T1w_class-wm_dtissue.nii.gz"
maskfilter: [100%] uncompressing image "derivatives/preprocess/sub-00/anat/sub-00_var-fsl_space-T1w_class-wm_dtissue.nii.gz"
maskfilter: [100%] applying erode filter to image sub-00_var-fsl_space-T1w_class-wm_dtissue.nii.gz
mrmath: [100%] uncompressing image "derivatives/preprocess/sub-00/anat/sub-00_var-fsl_space-T1w_class-wm_dtissue.nii.gz"
mrmath: [100%] computing product across 2 images
mrthreshold: [100%] thresholding "./mrtrix-tmp-YvnKYB.mif" at intensity 0.69999999999999996


In [ ]:
!dwi2response tournier {dwi_dir}/{dwi_src} \
    -lmax 6 \
    -mask {dname}/{sf} \
    {dname}/{rf}

In [ ]:
!dwi2fod csd {dwi_dir}/{dwi_src} \
    {dname}/{rf} \
    -lmax 6 \
    -mask {wmm_dir}/{wmm_src} \
    {dname}/{odf}

In [ ]:
!tckgen {dname}/{odf} {dname}/{tck} \
    -algorithm SD_PROB \
    -step $STEP \
    -act ${TT5} \
    -crop_at_gmwmi \
    -seed_dynamic ${ODF} \
    -maxlength 250 \
    -cutoff 0.1 \
    -number 10000


In [68]:
import os

In [74]:
ss = os.listdir('.')
print ss
ss.sort()
ss

['sub-00', 'input', 'derivatives']


['derivatives', 'input', 'sub-00']

In [ ]:
main